In [ ]:
# ==============================================================================
# ✅ [1차 학습 Step 1] 환경 설정 및 드라이브 마운트
# ==============================================================================
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets, models
import numpy as np
from tqdm.notebook import tqdm
import time
import glob
from collections import Counter
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from torch.utils.data.sampler import WeightedRandomSampler
import random
import copy

print("✅ 필수 라이브러리 로드 완료")

# Google Colab 환경에서 Google Drive 마운트
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive 마운트 완료")
except ImportError:
    print("⚠️ Google Colab 환경이 아닙니다. 로컬 환경에서 실행하는 경우, 드라이브 마운트가 필요 없습니다.")

# 디바이스 설정 (GPU 우선 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"⚙️ 현재 사용 디바이스: {device}")

In [ ]:
# ==============================================================================
# ✅ [1차 학습 Step 2] 데이터 로딩 및 불균형 처리 (Class Weight & Oversampling)
# ==============================================================================
# 🚨🚨🚨 사용자 데이터 폴더 경로 설정 (Google Drive 경로로 수정 완료) 🚨🚨🚨
data_dir = "/content/drive/MyDrive/busanit-251001/2-CNN/class3"

if not os.path.exists(data_dir):
    raise FileNotFoundError(f"⚠️ 경로를 찾을 수 없습니다: {data_dir}. Google Drive 경로를 다시 확인해주세요.")

# EfficientNet B0를 위한 전처리 설정
input_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# 데이터 증강 Transform
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

print("데이터셋 로딩 중...")

# 전체 데이터셋 로딩
full_dataset = datasets.ImageFolder(data_dir)
class_names = full_dataset.classes
num_classes = len(class_names)

# 훈련:검증 데이터 분할 (80:20)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_data, val_data = torch.utils.data.random_split(full_dataset, [train_size, val_size])

print(f"✅ 클래스 목록: {class_names}")
print(f"🔢 전체 클래스 수: {num_classes}")
print(f"🖼️ 전체 이미지 수: {len(full_dataset)}")
print(f"📏 훈련 데이터 크기: {len(train_data)}개, 검증 데이터 크기: {len(val_data)}개")

# Custom Dataset 클래스: 분할된 데이터에 전처리 적용
class CustomDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        original_index = self.subset.indices[index]
        x, y = self.subset.dataset[original_index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

# 전처리 적용된 데이터셋
image_datasets = {
    'train': CustomDataset(train_data, transform=data_transforms['train']),
    'val': CustomDataset(val_data, transform=data_transforms['val'])
}

# ----------------------------------------------------
# 🚨 불균형 처리 1: 클래스별 가중치 계산 (Loss Weighting)
# ----------------------------------------------------
train_labels = [full_dataset.targets[i] for i in train_data.indices]
class_counts = Counter(train_labels)
print(f"🔎 훈련 데이터 클래스별 개수: {class_counts}")

# 각 클래스의 가중치 (총 샘플 수 / 클래스별 샘플 수)
num_samples = len(train_labels)
class_weights = [num_samples / class_counts[i] for i in range(num_classes)]
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"⚖️ 손실 함수에 적용될 클래스 가중치: {class_weights_tensor.cpu().numpy()}")


# ----------------------------------------------------
# 🚨 불균형 처리 2: 오버샘플링을 위한 WeightedRandomSampler 적용
# ----------------------------------------------------
sample_weights = [class_weights[label] for label in train_labels]
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)
print("♻️ 훈련 데이터에 WeightedRandomSampler (오버샘플링) 적용 완료.")


# 데이터로더 설정
batch_size = 32
dataloaders_dict = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, sampler=sampler, num_workers=2),
    'val': DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=False, num_workers=2)
}

In [ ]:
# ==============================================================================
# ✅ [1차 학습 Step 3] EfficientNet-B0 모델 로드 및 전이 학습 설정
# ==============================================================================
print("\n EfficientNet-B0 모델 로드 중...")
model_ft = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# 특징 추출 레이어 동결 (Freeze)
for param in model_ft.parameters():
    param.requires_grad = False

# 최종 분류 레이어 (Classifier) 변경
num_ftrs = model_ft.classifier[1].in_features
model_ft.classifier[1] = nn.Linear(num_ftrs, num_classes)

model_ft = model_ft.to(device)

print("✅ 모델 설정 완료")
print(f"최종 분류 클래스 수: {num_classes}")

In [ ]:
# ==============================================================================
# ✅ [1차 학습 Step 4] 모델 훈련 함수 및 실행 (고급 전략 적용)
# ==============================================================================
# 🚨 손실 함수 설정 시 class_weights 적용
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
# 1차 학습은 마지막 분류 레이어만 학습 (Freeze 해제된 파라미터만)
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model_ft.parameters()), lr=0.001)

# ----------------------------------------------------
# 🚨 고급 데이터 증강: MixUp 구현 함수
# ----------------------------------------------------
MIXUP_ALPHA = 0.4

def mixup_data(x, y, alpha=1.0, device='cuda'):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, checkpoint_path='efficientnet_best_balanced.pth'):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_f1 = -1.0 # 🚨 최적 모델 기준을 Macro F1 Score로 설정

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            all_labels = []
            all_preds = []

            for inputs, labels in tqdm(dataloaders[phase], desc=f'{phase} Phase'):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train':
                        # 🚨 50% 확률로 MixUp 적용
                        if random.random() < 0.5:
                            mixed_inputs, targets_a, targets_b, lam = mixup_data(inputs, labels, MIXUP_ALPHA, device)
                            outputs = model(mixed_inputs)
                            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
                        else:
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                    else: # Validation Phase
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # 통계 계산
                    if phase == 'val':
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)


            epoch_loss = running_loss / len(image_datasets[phase])

            print(f'{phase} Loss: {epoch_loss:.4f}', end=' ')

            if phase == 'val':
                # 🚨 성능 모니터링: Macro F1 Score 및 Accuracy 계산
                current_acc = accuracy_score(all_labels, all_preds)
                current_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

                # 클래스별 정확도 계산
                cm = confusion_matrix(all_labels, all_preds)
                # zero_division=0 처리: 해당 클래스에 샘플이 없거나(sum=0) 예측이 0인 경우를 방지
                with np.errstate(divide='ignore', invalid='ignore'):
                    class_accuracy = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))

                print(f'Acc: {current_acc:.4f} Macro F1: {current_f1:.4f}')
                print('--- 클래스별 정확도 ---')
                for i, acc in enumerate(class_accuracy):
                    print(f'  {class_names[i]}: {acc:.4f}', end=' ')
                print('\n-----------------------')

                # 🚨 최적 모델 저장 기준: Macro F1 Score가 가장 높을 때 저장
                if current_f1 > best_val_f1:
                    best_val_f1 = current_f1
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save(best_model_wts, checkpoint_path)
                    print(f"⭐ 최적 모델 저장됨: {checkpoint_path} (Macro F1: {best_val_f1:.4f})")

    time_elapsed = time.time() - since
    print(f'\n🌟 훈련 완료! 총 소요 시간: {time_elapsed // 60:.0f}분 {time_elapsed % 60:.0f}초')
    print(f'🏆 최적 검증 Macro F1: {best_val_f1:.4f}')

    model.load_state_dict(best_model_wts)
    return model

# 훈련 실행
num_epochs = 50
model_ft = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, checkpoint_path='efficientnet_best_balanced.pth')

In [ ]:
# ==============================================================================
# ✅ [1차 학습 Step 5] 최종 모델 가중치 저장 (선택 사항)
# ==============================================================================
final_model_path = 'efficientnet_final_balanced.pth'
torch.save(model_ft.state_dict(), final_model_path)
print(f"✅ 최종 모델 가중치 저장 완료: {final_model_path}")

In [ ]:
# ==============================================================================
# 🧪 모델 예측 함수 정의 (1차 학습 후 테스트 시 재사용)
# ==============================================================================

def load_model_for_prediction(num_classes, model_path, device=device):
    # 모델 아키텍처 다시 로드
    model = models.efficientnet_b0(weights=None)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, num_classes)

    # 저장된 가중치 로드
    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        model.eval()
        print(f"✅ 모델 로드 완료: {model_path}")
    except Exception as e:
        print(f"❌ 가중치 파일을 로드할 수 없습니다. 경로 또는 파일명({model_path})을 확인하세요. 오류: {e}")
        return None
    return model

def preprocess_and_predict(image_path, model, transform, device):
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        print(f"⚠️  이미지 로드 중 오류 발생 ({image_path}): {e}")
        return -1, 0.0

    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_batch)
        probabilities = torch.nn.functional.softmax(output, dim=1)[0]
        confidence, predicted_class = torch.max(probabilities, 0)

    return predicted_class.item(), confidence.item()

# 예측용 전처리 (검증 데이터와 동일)
predict_transforms = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
# ==============================================================================
# 🧪 [1차 학습 후 테스트] - 1차 학습 최적 모델 테스트
# ==============================================================================
# 🚨🚨🚨 예측할 단일 이미지 파일의 정확한 경로를 여기에 입력하세요! 🚨🚨🚨
test_image_path_1st = "/content/drive/MyDrive/busanit-251001/2-CNN/후라이드1.jpg" # <--- 경로 수정 필요

print("\n"+"="*80)
print(f"🧪 1차 학습 최적 모델 테스트: {os.path.basename(test_image_path_1st)}")
print("="*80)

loaded_model_1st = load_model_for_prediction(num_classes, model_path='efficientnet_best_balanced.pth')

if loaded_model_1st:
    predicted_class, confidence = preprocess_and_predict(test_image_path_1st, loaded_model_1st, predict_transforms, device)

    if predicted_class != -1:
         pred_class = class_names[predicted_class]
         conf_percent = confidence * 100
         print(f"✅ 예측 클래스: {pred_class}")
         print(f"✅ 신뢰도: {conf_percent:.2f}%")
    else:
         print("⚠️ 예측 함수 실행 중 오류가 발생했습니다.")
print("="*80)

In [ ]:
# ==============================================================================
# 🧪 2차 학습: Fine-tuning 준비
# ==============================================================================
# 🚨 1차 학습 종료 후 이 셀부터 이어서 실행
# ----------------------------------------------------

# ✅ [2차 Fine-tuning Step 1] 모델 동결 해제 및 학습률 설정
print("\n✨ Fine-tuning 준비: 전체 모델 계층 동결 해제")

# 1차 학습에서 동결했던 전체 파라미터 해제
for param in model_ft.parameters():
    param.requires_grad = True

# 전체 파라미터에 대한 최적화 함수 재설정
# 미세 조정을 위해 1차 학습보다 훨씬 낮은 학습률(0.00001)을 사용합니다.
optimizer_ft_fine = optim.Adam(model_ft.parameters(), lr=0.00001)

# 🚨 2차 학습에서도 불균형 해소를 위해 Class Weights를 다시 적용합니다.
criterion_fine = nn.CrossEntropyLoss(weight=class_weights_tensor)


print("최초 10개 계층의 require_grad 상태 확인:")
for name, param in list(model_ft.named_parameters())[:10]:
    print(f"{name}: requires_grad={param.requires_grad}")

print("✅ 2차 Fine-tuning 준비 완료. 모든 계층이 학습됩니다. (Class Weights 적용)")


# ✅ [2차 Fine-tuning Step 2] 30 Epoch 추가 학습 실행
print("\n" + "="*80)
print("🚀 2차 Fine-tuning (추가 학습) 시작")
print("="*80)

try:
    # 1차 학습 시 저장된 최적의 균형 잡힌 모델 가중치를 로드합니다.
    model_ft.load_state_dict(torch.load('efficientnet_best_balanced.pth', map_location=device))
    print("✅ 1차 학습에서 저장된 최적 가중치로 Fine-tuning 시작!")
except FileNotFoundError:
    print("⚠️ 1차 학습 파일(efficientnet_best_balanced.pth)을 찾을 수 없습니다. 현재 모델 상태로 Fine-tuning을 진행합니다.")

num_epochs_fine = 30 # 추가 30회 훈련

model_ft_fine_tuned = train_model(
    model_ft,
    dataloaders_dict,
    criterion_fine,        # 🚨 수정된 손실 함수 사용
    optimizer_ft_fine,     # 🚨 수정된 최적화 함수 사용
    num_epochs=num_epochs_fine,
    checkpoint_path='efficientnet_fine_tuned_best_balanced.pth' # 2차 학습 전용 파일명으로 저장
)

print("✅ 2차 Fine-tuning 완료")

In [ ]:
# ==============================================================================
# 🧪 [2차 Fine-tuning 후 테스트] - 2차 학습 최적 모델 테스트
# ==============================================================================
# 🚨🚨🚨 예측할 단일 이미지 파일의 정확한 경로를 여기에 입력하세요! 🚨🚨🚨
test_image_path_2nd = "/content/drive/MyDrive/busanit-251001/2-CNN/후라이드1.jpg" # <--- 경로 수정 필요

print("\n"+"="*80)
print(f"🏆 2차 Fine-tuned 모델 최종 테스트: {os.path.basename(test_image_path_2nd)}")
print("="*80)

# 2차 학습에서 저장된 최적의 가중치 파일을 로드합니다.
loaded_model_2nd = load_model_for_prediction(num_classes, model_path='efficientnet_fine_tuned_best_balanced.pth')

if loaded_model_2nd:
    predicted_class_2nd, confidence_2nd = preprocess_and_predict(test_image_path_2nd, loaded_model_2nd, predict_transforms, device)

    if predicted_class_2nd != -1:
         pred_class_2nd = class_names[predicted_class_2nd]
         conf_percent_2nd = confidence_2nd * 100
         print(f"✅ 예측 클래스: {pred_class_2nd}")
         print(f"✅ 신뢰도: {conf_percent_2nd:.2f}%")
    else:
         print("⚠️ 예측 함수 실행 중 오류가 발생했습니다.")
print("="*80)